# Translate - Interpret
모델 설명을 위한 툴로 허깅페이스와 연동되는 아~주 유용한 XAI도구 입니다.

저는 여기서 Sequence Classification Explainer 를 사용하였습니다.

**출처 : https://github.com/cdpierse/transformers-interpret**


#Transformer translate

In [ ]:
# 우선 라이브러리를 설치하여 줍니다.
pip install transformers-interpret

     |████████████████████████████████| 1.4 MB 12.2 MB/s 
  Created wheel for transformers-interpret: filename=transformers_interpret-0.5.2-py3-none-any.whl size=23518 sha256=5723d8b847dc82051c783282e838122d4c6d3e5e2228f44dc727570e87dfe4b8
  Stored in directory: /root/.cache/pip/wheels/18/8a/5f/db643e1cd44e7bb22c21b52b8fcaebadafaf0b361c33f05869
Successfully built transformers-interpret


In [ ]:
"""
일단!!!
미리 만들어둔 모델들을 불러와야 합니다.
 
위의 Huggingface 모델에서 생성하고 저장 한 위치를 model과 tokenizer에 지정해주면 Auto기능 이 자동으로 해당
자료를 감지하여 가져옵니다. (신기합니다.)

"""
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer
#모델 불러오기 
model =  AutoModelForSequenceClassification.from_pretrained('./saved_2/klue-bert-base', from_tf=True) 
tokenizer = AutoTokenizer.from_pretrained("./saved_2/klue-bert-base")

#다중분류 설명자 생성
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)


All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
"""
여기 예시에서는 sample_data_v2를 사용하였는데 기존의 짧은 일반 대화문을 다른 혐오클래스와 같이 길게 재구성한것입니다.
"""
import pandas as pd
data = pd.read_csv('sample_data_v2.csv')
data.head()

print(data["class"].unique())

pd.set_option('display.max_columns', 100)
data_4 = data[data["class"] == 4]
data_4.head()

[0 1 2 3 4]


,Unnamed: 0,idx,class,conversation,pre_conversation
3584,7514,3564,4,"우엉차 있나요?\n 네, 오늘 한 거예요\n 우엉차 3000원짜리 말고 더 많은 ...",우엉차 있나요 네 오늘 한 거예요 우엉차 3000원짜리 말고 더 많은 양 있나요 그...
3585,8019,4069,4,파운데이션 사려고 하는데요\n 찾으시는 제품 있으신가요?\n 제품 추천 좀 해주세요...,파운데이션 사려고 하는데요 찾으시는 제품 있으신가요 제품 추천 좀 해주세요 이 제품...
3586,6940,2990,4,5시간 충전해주세요\n 충전되셨어요\n 정액제로 바꿀 수 있나요?\n 바꿔드릴게요\...,5시간 충전해주세요 충전되셨어요 정액제로 바꿀 수 있나요 바꿔드릴게요 라면 파나요 ...
3587,5660,1710,4,지금 바로 될까요\n 어떻게 잘라드릴까요\n 일단 앞머리는 눈썹 위로해주세요\n 조...,지금 바로 될까요 어떻게 잘라드릴까요 일단 앞머리는 눈썹 위로해주세요 조금만 잘라드...
3588,6823,2873,4,여기 아파트 월세가 있어요?\n 몇 평 찾으시는데요?\n 스물 아홉평인가\n #주소...,여기 아파트 월세가 있어요 몇 평 찾으시는데요 스물 아홉평인가 주소 스물 아홉평 월...


In [ ]:
"""
fake_sample_text와 general_sample_text 제가 임의로 생성한 문장입니다. 
fake_sample_text는 실제 혐오 문장이 아니지만 관련 단어들이 많이 들어 모델이 헷갈릴 만한 자료를 만들어 테스트 하기 위함이고
general_sample_text는 정말 일반 테스트를 위함입니다.

"""

fake_sample_text="""안녕하세요 저는 송영석 입니다. 자기소개를 시작하겠습니다. 저는 어릴적 강도질과 폭력을 일 삼았고 학창시절에는 나쁜 일들을 많이 저질렀습니다. 사람을 죽이고 싶었던 적이 많고 물건도 다 훔치고 빼았았습니다. 저는 죄를 반성하고 있습니다 """

In [ ]:
general_sample_text = """김 부사장은 삼성전자가 현재 만들고 있는 반도체들의 구조 변화와 전망을 개괄적으로 설명했습니다. 삼성전자 주력인 D램, 낸드플래시 등 메모리 반도체부터 이미지센서, 시스템 반도체 구조와 미래까지 개괄적으로 조망한 점이 눈에 띕니다. """

In [ ]:
"""
가져온 자료나 만든 문장을 multiclass_explainer안의 text에 지정해주면 visualize 되어지는 것을 확인 할 수 있습니다.
"""
#혐오 대화로 의심되어질 수 있는 일반 대화 test
word_attributions = multiclass_explainer(text=fake_sample_text)
html = multiclass_explainer.visualize()


In [ ]:
# 점수가 높을 수록 해당 클래스가 나오게 하는데 기여를 많이한 부분이고 
# 점수가 낮을 수록 관여를 적게한 부분입니다.

word_attributions

[('[CLS]', 0.0),
 ('안녕', -0.14508092883995555),
 ('##하', 0.049531531985935066),
 ('##세요', -0.04062209462564177),
 ('저', -0.06534830989339072),
 ('##는', -0.07334721621047807),
 ('송영', -0.3202626981429598),
 ('##석', 0.16263405626195007),
 ('입니다', 0.22061062453406174),
 ('.', -0.004663182660779118),
 ('자기', -0.054427734248404844),
 ('##소', 0.0803317363380772),
 ('##개', 0.1575229076937772),
 ('##를', 0.12337583974243611),
 ('시작', 0.1353114006862934),
 ('##하', 0.12261801677459254),
 ('##겠', 0.032140032233621316),
 ('##습', 0.08278307488204013),
 ('##니다', 0.11215853144076511),
 ('.', 0.0906373446245925),
 ('저', 0.04094830413126628),
 ('##는', 0.0781923348053056),
 ('어릴', 0.03690055716605718),
 ('##적', 0.17700596573725536),
 ('강도', 0.33502943718680234),
 ('##질', -0.04331879582663787),
 ('##과', 0.072036628214742),
 ('폭력', -0.04156015463763723),
 ('##을', 0.004146855214808768),
 ('일', 0.06003798384866029),
 ('삼', 0.1399011672930956),
 ('##았', 0.05962726338570945),
 ('##고', 0.031088054016330163),
 (

In [ ]:
#진짜 일반 대화 test
word_attributions = multiclass_explainer(text=general_sample_text)
html = multiclass_explainer.visualize()

In [ ]:
#문장을 짧게 짤라서도 테스트 해보았습니다. 이것은 기존의 sample_data에서 짧은 문장의 일반대화를 학습했었기 때문에 
#확인하기 위합입니다.

short_sample_text =[ """사람을 죽이고 싶었던 적이 많고 물건도 다 훔치고 빼았았습니다.""","""저는 어릴적 강도질과 폭력을 일 삼았고 학창시절에는 나쁜 일들을 많이 저질렀습니다.""","""안녕하세요 저는 송영석 입니다. 자기소개를 시작하겠습니다. """, """김 부사장은 삼성전자가 현재 만들고 있는 반도체들의 구조 변화와 전망을 개괄적으로 설명했습니다.""",
 """시스템 반도체 구조와 미래까지 개괄적으로 조망한 점이 눈에 띕니다."""]

In [ ]:
for i in range(0,len(short_sample_text)):
  word_attributions = multiclass_explainer(text=short_sample_text[i])
  html = multiclass_explainer.visualize()
  print("{0} 번째 문장의 클래스 : {1}".format(i,multiclass_explainer.predicted_class_name))


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,LABEL_0 (0.99),LABEL_0,3.54,[CLS] 사람 ##을 죽이 ##고 싶 ##었 ##던 적 ##이 많 ##고 물건 ##도 다 훔치 ##고 빼 ##았 ##았 ##습 ##니다 . [SEP]


0 번째 문장의 클래스 : LABEL_0


1 번째 문장의 클래스 : LABEL_0


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (0.80),LABEL_2,3.46,[CLS] 안녕 ##하 ##세요 저 ##는 송영 ##석 입니다 . 자기 ##소 ##개 ##를 시작 ##하 ##겠 ##습 ##니다 . [SEP]


2 번째 문장의 클래스 : LABEL_2


3 번째 문장의 클래스 : LABEL_2


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (0.48),LABEL_2,2.74,[CLS] 시스템 반도체 구조 ##와 미래 ##까 ##지 개 ##괄 ##적으로 조망 ##한 점 ##이 눈 ##에 [UNK] . [SEP]


4 번째 문장의 클래스 : LABEL_2
